In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2 
import torchvision
from torchvision import datasets,transforms
from tqdm import tqdm
import cv2
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from PIL import Image
import os
import torch.nn.functional as F
import ast

In [ ]:
# config
LR = 1e-4
SPLIT = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
EPOCHS = 2
DATAPATH = '../input/global-wheat-detection'

In [ ]:
!ls '../input/global-wheat-detection'

In [ ]:
df = pd.read_csv(DATAPATH + '/train.csv')
df.bbox = df.bbox.apply(ast.literal_eval)

In [ ]:
df = df.groupby("image_id")["bbox"].apply(list).reset_index(name="bboxes")

In [ ]:
def train_test_split(dataFrame,split):
    len_tot = len(dataFrame)
    val_len = int(split*len_tot)
    train_len = len_tot-val_len
    train_data,val_data = dataFrame.iloc[:train_len][:],dataFrame.iloc[train_len:][:]
    return train_data,val_data

In [ ]:
train_data_df,val_data_df = train_test_split(df,SPLIT)

In [ ]:
train_data_df

In [ ]:
class WheatDataset(Dataset):
    def __init__(self,data,root_dir,transform=None,train=True):
        self.data = data
        self.root_dir = root_dir
        self.image_names = self.data.image_id.values
        self.bboxes = self.data.bboxes.values
        self.transform = transform
        self.isTrain = train
    def __len__(self):
        return len(self.data)
    def __getitem__(self,index):
#         print(self.image_names)
#         print(self.bboxes)
        img_path = os.path.join(self.root_dir,self.image_names[index]+".jpg")
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        bboxes = torch.tensor(self.bboxes[index],dtype=torch.float64)
#         print(bboxes)
        """
            As per the docs of torchvision
            we need bboxes in format (xmin,ymin,xmax,ymax)
            Currently we have them in format (xmin,ymin,width,height)
        """
        bboxes[:,2] = bboxes[:,0]+bboxes[:,2]
        bboxes[:,3] = bboxes[:,1]+bboxes[:,3]
#         print(image.size,type(image))
        """
            we need to return image and a target dictionary
            target:
                boxes,labels,image_id,area,iscrowd
        """
        area = (bboxes[:,3]-bboxes[:,1])*(bboxes[:,2]-bboxes[:,0])
        area = torch.as_tensor(area,dtype=torch.float32)
        
        # there is only one class
        labels = torch.ones((len(bboxes),),dtype=torch.int64)
        
        # suppose all instances are not crowded
        iscrowd = torch.zeros((len(bboxes),),dtype=torch.int64)
        
        target = {}
        target['boxes'] = bboxes
        target['labels']= labels
        target['image_id'] = torch.tensor([index])
        target["area"] = area
        target['iscrowd'] = iscrowd
        
        if self.transform is not None:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transform(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
            
        return image,target
        

In [ ]:
train_transform = A.Compose([
    A.Flip(0.5),
    ToTensorV2(p=1.0)
],bbox_params = {'format':"pascal_voc",'label_fields': ['labels']})
val_transform = A.Compose([
      ToTensorV2(p=1.0)
],bbox_params = {'format':"pascal_voc","label_fields":['labels']})


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_data = WheatDataset(train_data_df,DATAPATH+"/train",transform=train_transform)
valid_data = WheatDataset(val_data_df,DATAPATH+"/train",transform=val_transform)

In [ ]:
image,target = train_data.__getitem__(0)
# plt.imshow(image)
print(image.shape)

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total=0.0
        self.iterations = 0.0
    def send(self,value):
        self.current_total+=value
        self.iterations+=1
    
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0*self.current_total/self.iterations
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
train_dataloader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,collate_fn=collate_fn)
val_dataloader = DataLoader(valid_data,batch_size=BATCH_SIZE,shuffle=False,collate_fn=collate_fn)

In [ ]:
train_loss = []
# val_loss = []
model = model.to(DEVICE)
params =[p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(params,lr=LR)
loss_hist = Averager()
itr = 1
lr_scheduler=None

In [ ]:
loss_hist = Averager()
itr = 1

for epoch in range(EPOCHS):
    loss_hist.reset()
    
    for images, targets in train_dataloader:
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")  

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')

In [ ]:
images, targets = next(iter(val_dataloader))
images = list(img.to(DEVICE) for img in images)
# print(images[0].shape)
targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
boxes = targets[1]['boxes'].cpu().numpy().astype(np.int32)
sample = images[1].permute(1,2,0).cpu().numpy()

In [ ]:
model.eval()
cpu_device = torch.device("cpu")
# print(images[0].shape)
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
# print(outputs)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
test_imgs_paths = pd.read_csv(os.path.join(DATAPATH,'sample_submission.csv'))
test_img_paths = test_imgs_paths.image_id.values
test_dir = DATAPATH+"/test"

In [ ]:
predictions=[]
image_ids=[]
with torch.no_grad():
    for path in test_img_paths:
        img_path = os.path.join(test_dir,path+".jpg")
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = np.transpose(image,(2,0,1))
    #     print(image.shape)
        image = torch.tensor(image,dtype=torch.float)
        image = image.unsqueeze(0)
    #     print(image.shape)
        image = image.to(DEVICE)
        outputs = model(image)

        predict=[]
        outputs = outputs[0]
        for i in range(len(outputs['boxes'])):
            temp = np.array([str(outputs['scores'][i].item()),str(outputs['boxes'][i][0].item()),str(outputs['boxes'][i][1].item()),str(outputs['boxes'][i][2].item()-outputs['boxes'][i][0].item()),str(outputs['boxes'][i][3].item()-outputs['boxes'][i][1].item())])
            predict.append(temp)
        predict = np.array(predict).flatten()
        predict = ' '.join(predict.flatten())
        image_ids.append(path)
        predictions.append(predict)
    print("------------Generating Submission File---------")
    df = pd.DataFrame({"image_id":image_ids,"PredictionString":predictions})
    df.to_csv('./submission.csv.gz',index=False,compression='gzip')